In [1]:
#Setting up the requirements
apikey = '' #Correspondiente a binance
secret = '' #Correspondinte a binance

In [2]:
##Librerias necesarias
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd
import numpy as np
import networkx as nx

In [3]:
#iniciamos nuestro cliente de binance
client = Client(apikey, secret) #conectamos nuestra cuenta de binance

In [4]:
###vamos a obtener los datos para nuestro dataframe y los transformamos en un dataframe
tipo_cambio = client.get_all_tickers()
tipo_cambio_df = pd.DataFrame(tipo_cambio)
tipo_cambio_df.set_index('symbol', inplace=True)

In [135]:
tipo_cambio_df

,price
symbol,
ETHBTC,0.08344700
LTCBTC,0.00328100
BNBBTC,0.01164600
NEOBTC,0.00060000
QTUMETH,0.00276500
...,...
SANDAUD,8.55900000
SLPBIDR,608.70
ANYBTC,0.00036920


In [5]:
#marcamos los tickers que vamos a utilizar
names = ['XRP', 'BTC', 'ETH', 'USDT', 'USDC', 'ADA', 'BNB', 'LTC', 'NEO', 'SOL']

In [6]:
table = np.zeros((len(names),len(names))) # creamos una matriz vacia con el tamaño que calce el numero de tickers
np.fill_diagonal(table, 1) # dado que es una matriz de tipo de cambio la diagonal debe ser de unos (debemos llearla nosotros porque en los datos no existen pares identicos)
tabla_df = pd.DataFrame(data = table, index = names, columns = names)  #creamos la tabla en un data frame

In [7]:
# Para ingresar los datos en el dataframe (tabla_df) debemos buscar la combinacion de tickers y asignarnos a los valores correpsondiente, ademàs 
# Debido a que no podemos saber el orden de los ticker debemos usar una combinatoria
# (Queda pendiete buscar alternativas màs eficientes)

def crear_tabla_tipo_cambio():
    for i in range(len(names)):
        
        for a in range(len(names)):
            
            for b in range(len(tipo_cambio_df.index)):
                
                if (tabla_df.columns[i] + tabla_df.index[a]) == tipo_cambio_df.index[b]:
                    tabla_df.loc[tabla_df.index[a], tabla_df.columns[i]] = tipo_cambio_df.iloc[b][0]
                
                elif (tabla_df.index[a] + tabla_df.columns[i]) == tipo_cambio_df.index[b]:
                    tabla_df.loc[tabla_df.columns[i], tabla_df.index[a]] = tipo_cambio_df.iloc[b][0]
        
                else:
                    continue
                    
# debido al orden debemos completar la simetria de la tabla.
def completar_tabla():
    for i in range(len(names)):
        
        for a in range(len(names)):
            
            if (tabla_df.iloc[i,a] == 0 and tabla_df.iloc[a,i] != 0):
                tabla_df.iloc[i,a] = tabla_df.iloc[a,i]
            elif (tabla_df.iloc[a,i] == 0 and tabla_df.iloc[i,a] != 0):
                tabla_df.iloc[a,i] = tabla_df.iloc[i,a]
            
            else:
                continue
                        

### Seccion de Algoritmos y graph theory para calcular las oportunidades de ar

In [8]:
# llamamos a las funciones
crear_tabla_tipo_cambio()
completar_tabla()

In [9]:
tabla_df

,XRP,BTC,ETH,USDT,USDC,ADA,BNB,LTC,NEO,SOL
XRP,1.0,0.00001652,0.00019810,0.79840000,0.79960000,0.0,0.00141800,0.0,0.0,0.0
BTC,0.00001652,1.0,0.08344700,48319.66000000,48373.79000000,0.00002918,0.01164600,0.00328100,0.00060000,0.00420690
ETH,0.00019810,0.08344700,1.0,4032.12000000,4036.07000000,0.00034980,0.13960000,0.03934000,0.00719000,0.05044000
USDT,0.79840000,48319.66000000,4032.12000000,1.0,0.99860000,1.41000000,562.70000000,158.50000000,29.03000000,203.31000000
USDC,0.79960000,48373.79000000,4036.07000000,0.99860000,1.0,1.41200000,562.90000000,158.50000000,28.90000000,203.69000000
ADA,0.0,0.00002918,0.00034980,1.41000000,1.41200000,1.0,0.00250500,0.0,0.0,0.0
BNB,0.00141800,0.01164600,0.13960000,562.70000000,0.21755000,0.00250500,1.0,0.28190000,0.05160000,0.36100000
LTC,0.0,0.00328100,0.03934000,158.50000000,158.50000000,0.0,0.28190000,1.0,0.0,0.0
NEO,0.0,0.00060000,0.00719000,29.03000000,28.90000000,0.0,0.05160000,0.0,1.0,0.0
SOL,0.0,0.00420690,0.05044000,203.31000000,203.69000000,0.0,0.36100000,0.0,0.0,1.0


In [10]:
# los valores estan en formato obj, necesitamos pasarlos a float()
tabla_df = tabla_df._convert(numeric=True)

In [37]:
##vamos a trabajar con el shortest path de ballman y ford por ende, pare mejorar la eficiencia
#y no hacer muchas multiplicaciones vamos a crear una tabla con los valores en log() (asi podemos sumar y no multiplicar)

edge_list_FX = []
nodes_list_FX = names

def extraer_edge_list_log(df, edge_list):
    for row in range(len(df)): #estamo trabajando con una matriz cuadrada
        for column in range(len(df)):
            edge_list.append((df.index[row], df.columns[column], -np.log(df.iloc[row, column]))) #creamos un tuple de la forma (tikr1, tikr2, pesos de su conexion que corresponde al tipo de cambio en forma log())
            
def extraer_edge_list(df, edge_list):
    for row in range(len(df)): #estamo trabajando con una matriz cuadrada
        for column in range(len(df)):
            edge_list.append((df.index[row], df.columns[column], (df.iloc[row, column]))) #sin el log para poder realizar algunas operaciones

In [141]:
edge_list_FX

[('XRP', 'XRP', 1.0),
 ('XRP', 'BTC', 1.652e-05),
 ('XRP', 'ETH', 0.0001981),
 ('XRP', 'USDT', 0.7984),
 ('XRP', 'USDC', 0.7996),
 ('XRP', 'ADA', 0.0),
 ('XRP', 'BNB', 0.001418),
 ('XRP', 'LTC', 0.0),
 ('XRP', 'NEO', 0.0),
 ('XRP', 'SOL', 0.0),
 ('BTC', 'XRP', 1.652e-05),
 ('BTC', 'BTC', 1.0),
 ('BTC', 'ETH', 0.083447),
 ('BTC', 'USDT', 48319.66),
 ('BTC', 'USDC', 48373.79),
 ('BTC', 'ADA', 2.918e-05),
 ('BTC', 'BNB', 0.011646),
 ('BTC', 'LTC', 0.003281),
 ('BTC', 'NEO', 0.0006),
 ('BTC', 'SOL', 0.0042069),
 ('ETH', 'XRP', 0.0001981),
 ('ETH', 'BTC', 0.083447),
 ('ETH', 'ETH', 1.0),
 ('ETH', 'USDT', 4032.12),
 ('ETH', 'USDC', 4036.07),
 ('ETH', 'ADA', 0.0003498),
 ('ETH', 'BNB', 0.1396),
 ('ETH', 'LTC', 0.03934),
 ('ETH', 'NEO', 0.00719),
 ('ETH', 'SOL', 0.05044),
 ('USDT', 'XRP', 0.7984),
 ('USDT', 'BTC', 48319.66),
 ('USDT', 'ETH', 4032.12),
 ('USDT', 'USDT', 1.0),
 ('USDT', 'USDC', 0.9986),
 ('USDT', 'ADA', 1.41),
 ('USDT', 'BNB', 562.7),
 ('USDT', 'LTC', 158.5),
 ('USDT', 'NEO', 29

In [38]:
extraer_edge_list(tabla_df,edge_list_FX)

##crear un objeto nx.graph
G_FX = nx.Graph()
G_FX.add_weighted_edges_from(edge_list_FX) #añadimos los nodos, los edges y los pesos a la grafica.

### Vamos a trabajar de dos maneras
### 1.  la primera es usando networkx.algorithms.cycles.minimum_cycle_basis para ver los cycles y posteriormente calcular cuales tienen oportunidades de arbitrage

In [133]:
## 1.networkx.algorithms.cycles.minimum_cycle_basis
# cycles = nx.algorithms.cycles.minimum_cycle_basis(G_FX)

ARB_op = [] # variable auxiliar

def search_negative_cycles_log_sum(G): #nos devuelve una lista con los nodos que poseen una oportunidad de arbitrage con los pesos en modo log
    cycle_list = nx.algorithms.cycles.minimum_cycle_basis(G)
    for i in cycle_list:
        if len(i) > 2:
            cycle_sum = 0
            cycle_sum = [G.get_edge_data(i[b], i[b+1])['weight'] if (b < len(i)-1) else G.get_edge_data(i[-1], i[0])['weight'] for b in range(len(i))]
            cycle_sum = np.sum(cycle_sum)
            if cycle_sum < 0:
                ARB_op.append((cycle_sum, i))
                
        else:
            continue
       
        
        
def search_positive_cycles_mul(G): #nos devuelve una lista con los nodos que poseen una oportunidad de arbitrage sin log
    cycle_list = nx.algorithms.cycles.minimum_cycle_basis(G)
    for i in cycle_list:
        if len(i) > 2:
            cycle_mul = 0
            cycle_mul = [G.get_edge_data(i[b], i[b+1])['weight'] if (b < len(i)-1) else G.get_edge_data(i[-1], i[0])['weight'] for b in range(len(i))]
            print(cycle_mul, i)
            cycle_mul = np.prod(cycle_mul)
            if cycle_mul > 1:
                ARB_op.append((cycle_mul, i))
                
        else:
            continue

In [134]:
##revisamos con la funcion las oportunidades de arbitrage
search_positive_cycles_mul(G_FX)

[1.652e-05, 0.083447, 0.0001981] ['XRP', 'BTC', 'ETH']
[1.652e-05, 48319.66, 0.7984] ['XRP', 'BTC', 'USDT']
[1.652e-05, 48373.79, 0.7996] ['XRP', 'BTC', 'USDC']
[1.652e-05, 2.918e-05, 0.0] ['XRP', 'BTC', 'ADA']
[1.652e-05, 0.011646, 0.001418] ['XRP', 'BTC', 'BNB']
[1.652e-05, 0.003281, 0.0] ['XRP', 'BTC', 'LTC']
[1.652e-05, 0.0006, 0.0] ['XRP', 'BTC', 'NEO']
[1.652e-05, 0.0042069, 0.0] ['XRP', 'BTC', 'SOL']
[0.0001981, 4032.12, 0.7984] ['XRP', 'ETH', 'USDT']
[0.0001981, 4036.07, 0.7996] ['XRP', 'ETH', 'USDC']
[0.0001981, 0.0003498, 0.0] ['XRP', 'ETH', 'ADA']
[0.0001981, 0.1396, 0.001418] ['XRP', 'ETH', 'BNB']
[0.0001981, 0.03934, 0.0] ['XRP', 'ETH', 'LTC']
[0.0001981, 0.00719, 0.0] ['XRP', 'ETH', 'NEO']
[0.0001981, 0.05044, 0.0] ['XRP', 'ETH', 'SOL']
[0.7996, 0.9986, 0.7984] ['XRP', 'USDC', 'USDT']
[0.0, 1.41, 0.7984] ['XRP', 'ADA', 'USDT']
[0.7984, 562.7, 0.001418] ['XRP', 'USDT', 'BNB']
[0.7984, 158.5, 0.0] ['XRP', 'USDT', 'LTC']
[0.0, 29.03, 0.7984] ['XRP', 'NEO', 'USDT']
[0.7984, 2

### 2. Aplicacion del Algoritmo de Bellman Ford

In [ ]:
#CODIGO DESDE: "https://the-algorithms.com/es/algorithm/bellman-ford"

# from __future__ import annotations


# def print_distance(distance: list[float], src):
#     print(f"Vertex\tShortest Distance from vertex {src}")
#     for i, d in enumerate(distance):
#         print(f"{i}\t\t{d}")


# def check_negative_cycle(
#     graph: list[dict[str, int]], distance: list[float], edge_count: int
# ):
#     for j in range(edge_count):
#         u, v, w = (graph[j][k] for k in ["src", "dst", "weight"])
#         if distance[u] != float("inf") and distance[u] + w < distance[v]:
#             return True
#     return False


# def bellman_ford(
#     graph: list[dict[str, int]], vertex_count: int, edge_count: int, src: int
# ) -> list[float]:
#     """
#     Returns shortest paths from a vertex src to all
#     other vertices.
#     >>> edges = [(2, 1, -10), (3, 2, 3), (0, 3, 5), (0, 1, 4)]
#     >>> g = [{"src": s, "dst": d, "weight": w} for s, d, w in edges]
#     >>> bellman_ford(g, 4, 4, 0)
#     [0.0, -2.0, 8.0, 5.0]
#     >>> g = [{"src": s, "dst": d, "weight": w} for s, d, w in edges + [(1, 3, 5)]]
#     >>> bellman_ford(g, 4, 5, 0)
#     Traceback (most recent call last):
#      ...
#     Exception: Negative cycle found
#     """
#     distance = [float("inf")] * vertex_count
#     distance[src] = 0.0

#     for i in range(vertex_count - 1):
#         for j in range(edge_count):
#             u, v, w = (graph[j][k] for k in ["src", "dst", "weight"])

#             if distance[u] != float("inf") and distance[u] + w < distance[v]:
#                 distance[v] = distance[u] + w

#     negative_cycle_exists = check_negative_cycle(graph, distance, edge_count)
#     if negative_cycle_exists:
#         raise Exception("Negative cycle found")

#     return distance


# if __name__ == "__main__":
#     import doctest

#     doctest.testmod()

#     V = int(input("Enter number of vertices: ").strip())
#     E = int(input("Enter number of edges: ").strip())

#     graph: list[dict[str, int]] = [dict() for j in range(E)]

#     for i in range(E):
#         print("Edge ", i + 1)
#         src, dest, weight = (
#             int(x)
#             for x in input("Enter source, destination, weight: ").strip().split(" ")
#         )
#         graph[i] = {"src": src, "dst": dest, "weight": weight}

#     source = int(input("\nEnter shortest path source:").strip())
#     shortest_distance = bellman_ford(graph, V, E, source)
#     print_distance(shortest_distance, 0)

In [137]:
mydict = [{'a': 1, 'b': 2, 'c': 3, 'd': 4},
          {'a': 100, 'b': 200, 'c': 300, 'd': 400},
          {'a': 1000, 'b': 2000, 'c': 3000, 'd': 4000 }]
df = pd.DataFrame(mydict)
df

,a,b,c,d
0,1,2,3,4
1,100,200,300,400
2,1000,2000,3000,4000


In [139]:
df.iloc[0,1]
df.iloc[row, column]

2